In [22]:
import pandas as pd


github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

df = df.iloc[:300]

In [23]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

In [24]:
answer_llm = df.iloc[0].answer_llm

In [29]:
vector = embedding_model.encode(answer_llm)

In [31]:
round(vector[0],2)

-0.42

In [48]:
def simi(record):
    answer_orig = record['answer_orig']
    answer_llm =  record['answer_llm']

    v_orig = embedding_model.encode(answer_orig)
    v_llm = embedding_model.encode(answer_llm)

    return v_llm.dot(v_orig)

results = df.to_dict(orient='records')

from tqdm import tqdm
simu = []

for record in tqdm(results_gpt4o):
    sim = simi(record)
    simu.append(sim)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:49<00:00,  6.01it/s]


In [49]:
df['cosine'] = simu

In [50]:
df['cosine'].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547923
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
Name: cosine, dtype: float64

In [51]:
import numpy as np
norm = np.sqrt(( df['cosine'] *  df['cosine']).sum())
v_norm = df['cosine'] / norm 

In [52]:
v_norm.describe()

count    300.000000
mean       0.056244
std        0.013060
min        0.009303
25%        0.049722
50%        0.057964
75%        0.064790
max        0.080749
Name: cosine, dtype: float64

In [39]:
!uv pip install rouge

Resolved 2 packages in 86ms                                          
Prepared 1 package in 10ms                                               
Installed 1 package in 4ms                                  
 + rouge==1.0.1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [53]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(df['answer_llm'][10], df['answer_orig'][10])[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [54]:
(scores['rouge-1']['f'] + scores['rouge-2']['f'] + scores['rouge-l']['f']) / 3

0.35490034990035496

In [55]:
scores = rouge_scorer.get_scores(df['answer_llm'], df['answer_orig'])[0]
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
print(rouge_avg)

0.07288173149369313
